In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import VarianceThreshold

import os
# Constants/filenames
gene_index = 4
genome_index = 1
cogfile = 'cogs_pfams_tigrfam_ko/cogs.txt'

def genomes_genes(file, genome_index, gene_index):
    unique_genomes = []
    unique_genes = []
    print('hi')
    with open(file, 'r+') as fh:
        for line in fh.readlines()[1:]:
            if line.split()[genome_index] not in unique_genomes:
                unique_genomes.append(line.split()[genome_index])
            if line.split()[gene_index] not in unique_genes:
                unique_genes.append(line.split()[gene_index])
    print("Done genes")
    return [unique_genomes, unique_genes]
    
def matrix(file, genes, genomes, gene_col, genome_col):
    gcols = pd.Index(genes, name = "genes")
    grows = pd.Index(genomes, name = "genomes")
    df = pd.DataFrame(data = np.zeros(shape = (len(genomes), len(genes))), index = grows, columns = gcols)
    fh = open(file, 'r+')
    for line in fh.readlines()[1:]:
        genome = line.split()[genome_index]
        gene = line.split()[gene_index]
        df.at[genome, gene] = 1
    fh.close()
    print('hi')
    return df
gng = genomes_genes(cogfile, genome_index, gene_index)
allgenomes = gng[0]
allgenes = gng[1]
feature_matrix = matrix(cogfile, allgenes, allgenomes, gene_index, genome_index)
feature_matrix['Y'] = [0] * len(feature_matrix)
print(feature_matrix.head(10))
print(len(feature_matrix))



hi
Done genes
hi
genes       COG1522  COG0859  COG0483  COG1066  COG0034  COG2217  COG0101  \
genomes                                                                     
2505679030      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
651053067       1.0      0.0      1.0      1.0      1.0      1.0      1.0   
637000240       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
637000208       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
650716076       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
637000053       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
637000192       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
651324080       1.0      0.0      1.0      1.0      1.0      1.0      1.0   
2571042743      1.0      0.0      1.0      1.0      1.0      1.0      1.0   
2574180005      1.0      0.0      1.0      1.0      1.0      1.0      1.0   

genes       COG0548  COG0642  COG1032  ...  COG5153  COG19

In [ ]:
print(feature_matrix.head(12))

In [18]:
import csv
with open("metadata_3837_genomes_june2016_complete.tsv") as tsvfile:
    reader = csv.DictReader(tsvfile, dialect = 'excel-tab')
    for ind, row in enumerate(reader):
        #if ind > 20: 
        #    break
        genome_id = row['taxon_oid']
        if genome_id in feature_matrix.index:
            # convert the OD to a taxon
            taxon = list(row.values())[4:-1]
            feature_matrix['Y'] = feature_matrix['Y'].astype(object)
            feature_matrix.at[genome_id, 'Y'] = taxon
#print(feature_matrix.iloc[0:9,1])
print(feature_matrix.head())

genes       COG1522  COG0859  COG0483  COG1066  COG0034  COG2217  COG0101  \
genomes                                                                     
2505679030      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
651053067       1.0      0.0      1.0      1.0      1.0      1.0      1.0   
637000240       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
637000208       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
650716076       1.0      1.0      1.0      1.0      1.0      1.0      1.0   

genes       COG0548  COG0642  COG1032  ...  COG5153  COG1967  COG1148  \
genomes                                ...                              
2505679030      1.0      1.0      0.0  ...      0.0      0.0      0.0   
651053067       1.0      1.0      1.0  ...      0.0      0.0      0.0   
637000240       1.0      1.0      1.0  ...      0.0      0.0      0.0   
637000208       1.0      1.0      1.0  ...      0.0      0.0      0.0   
650716076       1.0   

In [21]:
def genomicdistance(genome_id, reference_id):
# Recursive algorithm which checks for similarity
    def matching(taxon_1, taxon_2):
        
        if len(taxon_1) < 1 or len(taxon_2) < 1:
            return 0
        if taxon_1[0] == taxon_2[0]:
            return 1 + matching(taxon_1[1:], taxon_2[1:])
        else: 
            return 0
  #print(reference_id in feature_matrix.index)
    taxon_1 = feature_matrix.at[reference_id,'Y']
    taxon_2 = feature_matrix.at[genome_id, 'Y']
    #print(taxon_1)
    #print(taxon_2)
    if len(taxon_1) > len(taxon_2):
        return matching(taxon_1, taxon_2) / len(taxon_1)
    else:
        return matching(taxon_1, taxon_2) / len(taxon_2)


In [23]:
for genome, row in feature_matrix.iterrows():
    #print(type(genome))
    #print(genome)
    distance = genomicdistance(genome, '2505679030')
    feature_matrix.at[genome, 'Y2'] = distance
    #print(distance)
print(feature_matrix.iloc[450:500,-1])

genomes
2576861342    0.000000
2576861384    0.000000
2576861345    0.000000
2576861346    0.000000
2576861348    0.000000
2576861349    0.000000
2576861368    0.000000
2576861352    0.000000
2576861377    0.000000
2576861424    0.000000
2576861435    0.000000
2576861471    0.000000
2576861456    0.833333
2576861478    0.000000
2576861425    0.000000
2576861498    0.333333
2576861499    0.333333
2576861554    0.333333
640427122     0.000000
2576861487    0.000000
2576861488    0.000000
2576861656    0.000000
2576861677    0.000000
2576861679    0.000000
2576861765    0.000000
2576861768    0.000000
2576861772    0.000000
2576861745    0.000000
2576861748    0.000000
2576861752    0.000000
2576861754    0.000000
2576861755    0.000000
2576861758    0.000000
2576861779    0.000000
2576861725    0.000000
2576861726    0.000000
2576861707    0.000000
2576861731    0.000000
2576861732    0.000000
2576861737    0.000000
2576861669    0.000000
2576861740    0.000000
2571042112    0.000000
257

In [36]:
feature_matrix.to_csv("plant_gene_matrix.csv")

In [26]:
print(feature_matrix.head())

genes       COG1522  COG0859  COG0483  COG1066  COG0034  COG2217  COG0101  \
genomes                                                                     
2505679030      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
651053067       1.0      0.0      1.0      1.0      1.0      1.0      1.0   
637000240       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
637000208       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
650716076       1.0      1.0      1.0      1.0      1.0      1.0      1.0   

genes       COG0548  COG0642  COG1032  ...  COG1967  COG1148  COG3054  \
genomes                                ...                              
2505679030      1.0      1.0      0.0  ...      0.0      0.0      0.0   
651053067       1.0      1.0      1.0  ...      0.0      0.0      0.0   
637000240       1.0      1.0      1.0  ...      0.0      0.0      0.0   
637000208       1.0      1.0      1.0  ...      0.0      0.0      0.0   
650716076       1.0   

In [29]:

Y = pd.DataFrame(data = feature_matrix.iloc[:, -1], index = feature_matrix.index)
feature_matrix.drop(columns = ['Y', 'Y2'])
print(feature_matrix)

#print(Y)
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X, y = load_iris(return_X_y=True)



genes       COG1522  COG0859  COG0483  COG1066  COG0034  COG2217  COG0101  \
genomes                                                                     
2505679030      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
651053067       1.0      0.0      1.0      1.0      1.0      1.0      1.0   
637000240       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
637000208       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
650716076       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
...             ...      ...      ...      ...      ...      ...      ...   
2643221931      1.0      0.0      1.0      1.0      1.0      1.0      1.0   
2643221933      1.0      0.0      1.0      1.0      1.0      1.0      1.0   
2643221934      1.0      0.0      1.0      1.0      1.0      1.0      1.0   
2643221904      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
2643221912      1.0      1.0      1.0      1.0      1.0      1.0      1.0   

In [55]:

#print(feature_matrix)

# split feature matrix into features and target
X = feature_matrix.iloc[:, 0:-2]
#print(X)
Y = feature_matrix.iloc[:, -1]
#print(Y)
print("Originally had ", str(len(X.columns)), ' features')

# remove features with low variance, aka features with low correlation to a microbe's function/taxonomy
selector = VarianceThreshold(threshold=(.05)).fit(X,Y)

# selecting only most salient features,alternatively could use df.drop()
vdf = feature_matrix[feature_matrix.columns[selector.get_support(indices=True)]]

#print('V-based reduced feature matrix with ', str(len(vdf.columns)), 'features: \n', list(vdf.columns))
#print('\n from ', str(len(vdf)), ' genomes')

# update the target
Y = feature_matrix.loc[list(vdf.index), 'Y2']
print(Y)

X = vdf
print(X)




Originally had  4211  features
genomes
2505679030    1.0
651053067     0.0
637000240     0.0
637000208     0.0
650716076     0.0
             ... 
2643221931    0.0
2643221933    0.0
2643221934    0.0
2643221904    0.0
2643221912    0.0
Name: Y2, Length: 3837, dtype: float64
genes       COG0859  COG0483  COG1032  COG0559  COG1804  COG1492  COG3173  \
genomes                                                                     
2505679030      1.0      1.0      0.0      0.0      1.0      0.0      1.0   
651053067       0.0      1.0      1.0      1.0      1.0      1.0      1.0   
637000240       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
637000208       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
650716076       1.0      1.0      0.0      1.0      1.0      1.0      1.0   
...             ...      ...      ...      ...      ...      ...      ...   
2643221931      0.0      1.0      0.0      1.0      1.0      1.0      0.0   
2643221933      0.0      1.0   

In [38]:
# get the 10 best features correlated to genomiic distance?
#bestfeatures = SelectKBest(k=100)
#fit = bestfeatures.fit(X,Y)
#dfscores = pd.DataFrame(fit.scores_)
#dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
#featureScores = pd.concat([dfcolumns,dfscores],axis=1)
#featureScores.columns = ['Specs','Score']  #naming the dataframe columns
#print(featureScores.nlargest(100,'Score'))  #print 10 best features
#print(type(featureScores))
#print(featureScores)

In [1]:
import csv
import pandas as pd
#feature_matrix = pd.read_csv("plant_gene_matrix.csv")
#print(feature_matrix)
print(vdf)

NameError: name 'vdf' is not defined

In [47]:
print(vdf.head())

genes       COG0859  COG0483  COG1032  COG0559  COG1804  COG1492  COG3173  \
genomes                                                                     
2505679030      1.0      1.0      0.0      0.0      1.0      0.0      1.0   
651053067       0.0      1.0      1.0      1.0      1.0      1.0      1.0   
637000240       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
637000208       1.0      1.0      1.0      1.0      1.0      1.0      1.0   
650716076       1.0      1.0      0.0      1.0      1.0      1.0      1.0   

genes       COG0625  COG1638  COG1523  ...  COG4283  COG4245  COG3529  \
genomes                                ...                              
2505679030      0.0      0.0      0.0  ...      0.0      0.0      0.0   
651053067       1.0      1.0      1.0  ...      0.0      0.0      0.0   
637000240       1.0      1.0      1.0  ...      0.0      0.0      0.0   
637000208       1.0      1.0      0.0  ...      0.0      0.0      0.0   
650716076       1.0   

In [ ]:
# Correlation Matrix? yikes.
# If doing correlation, make sure that 'Y' is included in df
#import matplotlib.pyplot as plt
#import seaborn as sns
#Using Pearson Correlation
#plt.figure(figsize=(12,10))
#cor = feature_matrix.corr()
#sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
#plt.show()

In [ ]:
#UMAP time

import umap.umap_ as umap
reducer = umap.UMAP()
embedding = reducer.fit_transform(vdf)
embedding.shape
